In [51]:
!pip uninstall -y tensorboard
!pip uninstall -y tensorflow
!pip uninstall -y autoai-libs

!pip install -U geocoder
!pip install -U geopy
!pip install -U googlemaps
!pip install -U folium
!pip install -U numpy
!pip install -U pandas

import pandas as pd
import numpy as np

from pandas import json_normalize  # tranform JSON file into a pandas dataframe

import googlemaps
from datetime import datetime
import requests
import geopy

geopy.geocoders.options.default_user_agent = "my-application"

import ast
import json
import time

import folium
from folium.plugins import HeatMap

from ibm_botocore.client import Config
import ibm_boto3

print('Ready')

Requirement already up-to-date: geocoder in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.38.1)
Requirement already up-to-date: geopy in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.21.0)
Requirement already up-to-date: googlemaps in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.2.2)
Requirement already up-to-date: folium in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.10.1)
Requirement already up-to-date: numpy in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.18.3)
Requirement already up-to-date: pandas in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.0.3)
Ready


In [52]:
# The code was removed by Watson Studio for sharing.

In [53]:
!wget --cookies=on --save-cookies cookies.txt --keep-session-cookies --post-data 'user=EMAIL&password=PASSWORD' 'https://www.ctt.pt/fecas/login?service=https%3A%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx'
!wget --cookies=on --load-cookies cookies.txt --keep-session-cookies 'https://www.ctt.pt/feapl_2/app/restricted/postalCodeSearch/postalCodeDownloadFiles!downloadPostalCodeFile.jspx' -O PostCodeFull.Pt.zip

--2020-04-25 14:37:44--  https://www.ctt.pt/fecas/login?service=https%3A%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx
Resolving www.ctt.pt (www.ctt.pt)... 62.28.56.1
Connecting to www.ctt.pt (www.ctt.pt)|62.28.56.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10936 (11K) [text/html]
Saving to: ‘login?service=https:%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx.1’

100%[======================================>] 10,936      --.-K/s   in 0.04s   

2020-04-25 14:37:45 (241 KB/s) - ‘login?service=https:%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx.1’ saved [10936/10936]

--2020-04-25 14:37:46--  https://www.ctt.pt/feapl_2/app/restricted/postalCodeSearch/postalCodeDownloadFiles!downloadPostalCodeFile.jspx
Resolving www.ctt.pt (www.ctt.pt)... 62.28.56.1
Connecting to www.ctt.pt (www.ctt.pt)|62.28.56.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.ctt.pt/fecas/login?service=https%3

In [54]:
# The code was removed by Watson Studio for sharing.

In [55]:
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [56]:
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')

In [57]:
!ls -la

total 28260
drwxr-x--- 2 dsxuser dsxuser     4096 Apr 25 14:37 .
drwx------ 1 dsxuser dsxuser     4096 Apr 25 09:50 ..
-rw-r----- 1 dsxuser dsxuser     5703 Apr 25 10:35 concelhos.txt
-rw-r----- 1 dsxuser dsxuser      201 Apr 25 14:37 cookies.txt
-rw-r----- 1 dsxuser dsxuser      456 Apr 25 10:35 distritos.txt
-rw-r----- 1 dsxuser dsxuser  1216044 Apr 25 13:33 food_Hotels.csv
-rw-r----- 1 dsxuser dsxuser   283782 Apr 25 14:32 hotels_all.csv
-rw-r----- 1 dsxuser dsxuser   119358 Apr 25 10:35 hotels_Resort.csv
-rw-r----- 1 dsxuser dsxuser     2041 Apr 25 12:50 location_geometry.csv
-rw-r----- 1 dsxuser dsxuser    10936 Apr 25 11:48 login?service=https:%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx
-rw-r----- 1 dsxuser dsxuser    10936 Apr 25 14:37 login?service=https:%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx.1
-rw-r----- 1 dsxuser dsxuser   438008 Apr 25 13:39 nightlife_Hotels.csv
-rw-r----- 1 dsxuser dsxuser     9835 Apr 25 14:37 PostCodeFull.Pt.zip
-rw-r-

In [58]:
# This ZIP contains (distritos.txt, concelhos.txt, todos_cp.txt)
!unzip -x PostCodeFull.Pt.zip

Archive:  PostCodeFull.Pt.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of PostCodeFull.Pt.zip or
        PostCodeFull.Pt.zip.zip, and cannot find PostCodeFull.Pt.zip.ZIP, period.


In [59]:
df_DD=pd.read_csv('distritos.txt',sep=';',encoding='iso8859_15')
df_DD.head()

,DD,DESIG
0,1,Aveiro
1,2,Beja
2,3,Braga
3,4,Bragança
4,5,Castelo Branco


In [60]:
df_CC=pd.read_csv('concelhos.txt',sep=';',encoding='iso8859_15')
df_CC=df_CC[df_CC['DD']==8]
df_CC.head()

,DD,CC,DESIG
101,8,6,Lagoa (Algarve)
102,8,7,Lagos
103,8,8,Loulé
104,8,9,Monchique
105,8,10,Olhão


In [61]:
df_PostalCode=pd.read_csv('todos_cp.txt',sep=';',encoding='iso8859_15', index_col=False).reset_index()[['DD','CC','LLLL','LOCALIDADE','CP4','CP3','CPALF']]
df_PostalCode.head()

,DD,CC,LLLL,LOCALIDADE,CP4,CP3,CPALF
0,1,1,249,Alcafaz,3750,11,AGADÃO
1,1,1,250,Caselho,3750,12,AGADÃO
2,1,1,251,Corga da Serra,3750,13,AGADÃO
3,1,1,252,Foz,3750,14,AGADÃO
4,1,1,253,Guistola,3750,15,AGADÃO


In [62]:
# Select only location from Algarve
df_CPALF_Algarve=pd.DataFrame(df_PostalCode[df_PostalCode['DD']==8].groupby(['CPALF'])['CP4'].count()).reset_index()[['CPALF']]
df_CPALF_Algarve.head()

,CPALF
0,ALBUFEIRA
1,ALCANTARILHA
2,ALCOUTIM
3,ALFERCE
4,ALGOZ


In [63]:
# The code was removed by Watson Studio for sharing.

In [64]:
df_all_locations=pd.DataFrame()
for ind in df_CPALF_Algarve.index:
    address='{}, Algarve, Portugal'.format(df_CPALF_Algarve['CPALF'][ind])
    try:
        gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY,timeout=1)
        location = gmaps.geocode(address)
    except geopy.exc.GeocoderUnavailable:
        print('GeocoderUnavailable')
        print(name)
    except urllib.error.URLError:
        print('URLError')
        print(name)
    except OSError:
        print('OSError')
        print(name)


    if df_all_locations.empty:
        df_all_locations=pd.DataFrame(json_normalize(location))
    else:
        df_all_locations=df_all_locations.append(json_normalize(location),sort=False)

print(df_all_locations.shape)

(73, 18)


In [65]:
df_location_geometry=df_all_locations.reset_index()[['geometry.location.lat','geometry.location.lng']].reset_index()
df_location_geometry.to_csv('location_geometry.csv',index=False)
upload_file_cos(credentials,'location_geometry.csv','location_geometry.csv')

 File Uploaded


In [66]:
# Remove duplications
df_location_geometry_sorted=df_location_geometry.sort_values(by=['geometry.location.lat', 'geometry.location.lng'],ignore_index=True).reset_index()[['geometry.location.lat','geometry.location.lng']]
df_location_geometry_sorted.head()

,geometry.location.lat,geometry.location.lng
0,37.016829,-8.940593
1,37.019355,-7.930440
2,37.026042,-7.842351
3,37.047121,-7.744386
4,37.071629,-8.101987


In [67]:
# The code was removed by Watson Studio for sharing.

In [68]:
#category='4bf58dd8d48988d12f951735' # Resort
category='4bf58dd8d48988d1fa931735' # Hotel

df_all_hotels=pd.DataFrame()
for ind in df_location_geometry_sorted.index:

    latitude = df_location_geometry_sorted['geometry.location.lat'][ind]
    longitude = df_location_geometry_sorted['geometry.location.lng'][ind]

    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET,
        VERSION,
        latitude,
        longitude,
        9999,
        50,
        category)
    results = json_normalize(requests.get(url).json())
    try:
        if not results.empty:
            response_venues=results['response.venues'][0]
            if response_venues:
                if df_all_hotels.empty:
                    df_all_hotels=pd.DataFrame(response_venues)
                else:
                    df_all_hotels=df_all_hotels.append(response_venues,sort=False)
    except KeyError:
        print('Error')

print(len(df_all_hotels))

2785


In [69]:
# Remove duplications
df_all_hotels_sorted = df_all_hotels.sort_values(by=['name'], ignore_index=True).reset_index()[['id', 'location', 'name']]
df_all_hotels_sorted.drop_duplicates(subset=['id'],ignore_index=True,inplace=True)
df_all_hotels_sorted.head()

,id,location,name
0,5b2d6e07b2958f002c54e4ba,"{'lat': 37.090429, 'lng': -8.209487, 'labeledL...",1-211 appartement
1,4f7350c2e4b0257534d4008b,"{'address': 'Algarve', 'lat': 37.10418, 'lng':...",402
2,575882b6498e96a54155636e,"{'lat': 37.093356, 'lng': -8.743884, 'labeledL...",A Cabana B&B
3,5011b9abe4b0c3be96766adf,"{'lat': 37.09199419481045, 'lng': -8.209547670...",Acqua Maris
4,4da4481b2939b1f7049e2e57,"{'address': 'Rocha Baixinha', 'lat': 37.082724...",Adriana Beach Club & Spa


In [70]:
# Searching for all food venues
category='4d4b7105d754a06374d81259' # Food
df_all_food=pd.DataFrame()

for ind in df_all_hotels_sorted.index:

    location = df_all_hotels_sorted['location'][ind]
    try:
        coord=pd.DataFrame(location['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError: # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError: # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    try:
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            latitude,
            longitude,
            999,
            50,
            category)
        results = json_normalize(requests.get(url).json())

        if not results.empty:
            response_venues=results['response.venues'][0]
            if response_venues:
                if df_all_food.empty:
                    df_all_food=pd.DataFrame(response_venues)
                    df_all_hotels_sorted['food.count'] = 0
                else:
                    df_all_food=df_all_food.append(response_venues,sort=False)
                df_all_hotels_sorted.loc[ind, 'food.count'] = len(response_venues)

    except KeyError:
        print('Error')

print(df_all_food.shape)

(18655, 7)


In [78]:
df_all_food_sorted=df_all_food.sort_values(by=['name'],ignore_index=True).reset_index()[['id','location','name']]
df_all_food_sorted.drop_duplicates(subset=['id'],ignore_index=True,inplace=True)
df_all_food_sorted.shape

(3691, 3)

In [ ]:
df_all_food_sorted.head()

In [72]:
df_all_food_sorted.to_csv('food_Hotels.csv',index=False)
upload_file_cos(credentials,'food_Hotels.csv','food_Hotels.csv')

 File Uploaded


In [73]:
# Searching for all Nightlife venues
category = '4d4b7105d754a06376d81259'  # Nightlife Spot
df_all_Nightlife=pd.DataFrame()

for ind in df_all_hotels_sorted.index:

    location = df_all_hotels_sorted['location'][ind]
    try:
        coord=pd.DataFrame(location['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError: # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError: # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    try:
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            latitude,
            longitude,
            999,
            50,
            category)
        results = json_normalize(requests.get(url).json())

        if not results.empty:
            response_venues=results['response.venues'][0]
            if response_venues:
                if df_all_Nightlife.empty:
                    df_all_Nightlife=pd.DataFrame(response_venues)
                    df_all_hotels_sorted['nightlife.count'] = 0
                else:
                    df_all_Nightlife=df_all_Nightlife.append(response_venues,sort=False)
                df_all_hotels_sorted.loc[ind, 'nightlife.count'] = len(response_venues)

    except KeyError:
        print('Error')

print(df_all_Nightlife.shape)

(10254, 7)


In [79]:
df_all_Nightlife_sorted=df_all_Nightlife.sort_values(by=['name'],ignore_index=True).reset_index()[['id','location','name']]
df_all_Nightlife_sorted.drop_duplicates(subset=['id'],ignore_index=True,inplace=True)
df_all_Nightlife_sorted.shape

(1369, 3)

In [ ]:
df_all_Nightlife_sorted.head()

In [75]:
df_all_Nightlife_sorted.to_csv('nightlife_Hotels.csv',index=False)
upload_file_cos(credentials,'nightlife_Hotels.csv','nightlife_Hotels.csv')

 File Uploaded


In [76]:
df_all_hotels_sorted['food.ratio'] = df_all_hotels_sorted['food.count'] / 50
df_all_hotels_sorted['nightlife.ratio'] = df_all_hotels_sorted['nightlife.count'] / 50
df_all_hotels_sorted.head()

,id,location,name,food.count,nightlife.count,food.ratio,nightlife.ratio
0,5b2d6e07b2958f002c54e4ba,"{'lat': 37.090429, 'lng': -8.209487, 'labeledL...",1-211 appartement,30,17,0.60,0.34
1,4f7350c2e4b0257534d4008b,"{'address': 'Algarve', 'lat': 37.10418, 'lng':...",402,9,2,0.18,0.04
2,575882b6498e96a54155636e,"{'lat': 37.093356, 'lng': -8.743884, 'labeledL...",A Cabana B&B,10,2,0.20,0.04
3,5011b9abe4b0c3be96766adf,"{'lat': 37.09199419481045, 'lng': -8.209547670...",Acqua Maris,33,17,0.66,0.34
4,4da4481b2939b1f7049e2e57,"{'address': 'Rocha Baixinha', 'lat': 37.082724...",Adriana Beach Club & Spa,9,5,0.18,0.10


In [77]:
df_all_hotels_sorted.to_csv('hotels_all.csv',index=False)
upload_file_cos(credentials,'hotels_all.csv','hotels_all.csv')

 File Uploaded
